# Please run this .ipynb in Google Colab with an associated Google Drive account. You may want to consult the following two links to get more information about how to get FREE access to these resources.

https://www.google.com/drive/

https://colab.research.google.com/

## (i) Here we install the onnxruntime library

In [ ]:
!pip install onnxruntime

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# (ii) Here we import all necessary libraries

In [ ]:
from google.colab import drive
drive.mount("/content/drive")
import onnxruntime as rt
import numpy as np

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## (iii) Please put all your files at a desired location. Ideally, under "/content/drive/MyDrive/" folder. Just upload all your files to the root folder of Google Drive should work.

## (iv) Load all necessary files and use our models to make prediction

Model for males

In [ ]:
providers = ['CPUExecutionProvider']
output_path = "/content/drive/MyDrive/3dcnn-male.onnx" #Pls change the path and load our onnx model
output_names = ['dense_1']
m = rt.InferenceSession(output_path, providers=providers)
x = np.load("/content/drive/MyDrive/cn_males.npy")# This is the path for the input data. 
onnx_pred = m.run(output_names, {"input": x})

In [ ]:
y = np.load("/content/drive/MyDrive/y_cn_males.npy")# This is CA
print(y)
print(onnx_pred[0] + 22) # This is predicted BA. There is a bias of 22 to train the model. Now we add it back.

[79. 67. 66. 23. 57. 69. 67. 71. 28. 68.]
[[76.22479 ]
 [64.5877  ]
 [63.846954]
 [22.02018 ]
 [55.6043  ]
 [67.97568 ]
 [63.4654  ]
 [68.272835]
 [27.262028]
 [64.97217 ]]


Model for females

In [ ]:
providers = ['CPUExecutionProvider']
output_path = "/content/drive/MyDrive/3dcnn-female.onnx"
output_names = ['dense_1']
m = rt.InferenceSession(output_path, providers=providers)
x = np.load("/content/drive/MyDrive/cn_females.npy")
onnx_pred = m.run(output_names, {"input": x})

In [ ]:
y = np.load("/content/drive/MyDrive/y_cn_females.npy")
print(y)
print(onnx_pred[0] + 22) # There is a bias of 22 to train the model. Now we add it back.

[29. 23. 52. 66. 60. 72. 74. 57. 64. 48.]
[[30.903347]
 [26.209435]
 [53.66224 ]
 [65.57459 ]
 [63.46093 ]
 [71.929565]
 [72.86903 ]
 [57.87299 ]
 [63.05902 ]
 [51.093704]]
